In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Library


In [5]:
import functools
import math
import os  # noqa: F401
from random import choice, randint
from time import time

import numpy as np
import torch
import torch.utils.checkpoint as checkpoint
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from tqdm import tqdm

import json
from transformers import AdamW, AutoModel, AutoModelForSeq2SeqLM, AutoTokenizer,  get_linear_schedule_with_warmup
import pandas as pd

# Data

In [4]:
with open(r'/kaggle/input/eli5-10-doc/ELI5_train_10_doc.json') as f:
    train = json.load(f)
    
with open(r'/kaggle/input/eli5-10-doc/ELI5_val_10_doc.json') as f:
    val = json.load(f)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# pretrained DPR

In [ ]:
# qar_tokenizer = AutoTokenizer.from_pretrained('yjernite/retribert-base-uncased')
# qar_model = AutoModel.from_pretrained('yjernite/retribert-base-uncased').to('cuda:1')
# _ = qar_model.eval()

# Preprocessing data
1. Embedding passsage
2. Embedding question

In [ ]:
# def embed_passages_for_retrieval(passages, tokenizer, qa_embedder, max_length=128, device="cuda:1"):
#     a_toks = tokenizer.batch_encode_plus(passages, max_length=max_length, pad_to_max_length=True)
#     a_ids, a_mask = (
#         torch.LongTensor(a_toks["input_ids"]).to(device),
#         torch.LongTensor(a_toks["attention_mask"]).to(device),
#     )
#     with torch.no_grad():
#         a_reps = qa_embedder.embed_answers(a_ids, a_mask).cpu().type(torch.float)
#     return a_reps.numpy()

In [ ]:
# def embed_questions_for_retrieval(q_ls, tokenizer, qa_embedder, device="cuda:1"):
#     q_toks = tokenizer.batch_encode_plus(q_ls, max_length=128, pad_to_max_length=True)
#     q_ids, q_mask = (
#         torch.LongTensor(q_toks["input_ids"]).to(device),
#         torch.LongTensor(q_toks["attention_mask"]).to(device),
#     )
#     with torch.no_grad():
#         q_reps = qa_embedder.embed_questions(q_ids, q_mask).cpu().type(torch.float)
#     return q_reps.numpy()

In [ ]:
# sample = val[0]
# support_doc = "<P> " + " <P> ".join([str(p) for p in sample['ctxs']])
# support_doc

In [ ]:
# def format_dataset(dataset: list):
#     for sample in dataset:
#         sample['ctxs'] = "<P> " + " <P> ".join([p["passage_text"] for p in sample['ctxs']])
#     return dataset

In [ ]:
# val[0]

In [ ]:
# train_format = format_dataset(train[0:172634])
# val_format = format_dataset(val[0:5])
# val_format

In [ ]:
# for sample in val:
#     sample['ctxs'] = embed_passages_for_retrieval(sample['ctxs'], qar_tokenizer, qar_model)
# val[0]


# Training

In [5]:
class ArgumentsS2S():
    def __init__(self):
        self.batch_size = 8
        self.backward_freq = 16
        self.max_length = 1024
        self.print_freq = 100
        self.model_save_name = "seq2seq_models/eli5_bart_model"
        self.learning_rate = 2e-4
        self.num_epochs = 1

In [6]:
def make_qa_s2s_model(model_name="facebook/bart-large", from_file=None, device="cuda:0"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    if from_file is not None:
        param_dict = torch.load(from_file)  # has model weights, optimizer, and scheduler states
        model.load_state_dict(param_dict["model"])
    return tokenizer, model

In [7]:
def make_qa_s2s_batch(qa_list, tokenizer, max_len=64, max_a_len=360, device="cuda:0"):
    q_ls = [q for q, a in qa_list]
    a_ls = [a for q, a in qa_list]
    q_toks = tokenizer.batch_encode_plus(q_ls, max_length=max_len, pad_to_max_length=True)
    q_ids, q_mask = (
        torch.LongTensor(q_toks["input_ids"]).to(device),
        torch.LongTensor(q_toks["attention_mask"]).to(device),
    )
    a_toks = tokenizer.batch_encode_plus(a_ls, max_length=min(max_len, max_a_len), pad_to_max_length=True)
    a_ids, a_mask = (
        torch.LongTensor(a_toks["input_ids"]).to(device),
        torch.LongTensor(a_toks["attention_mask"]).to(device),
    )
    labels = a_ids[:, 1:].contiguous().clone()
    labels[a_mask[:, 1:].contiguous() == 0] = -100
    model_inputs = {
        "input_ids": q_ids,
        "attention_mask": q_mask,
        "decoder_input_ids": a_ids[:, :-1].contiguous(),
        "labels": labels,
    }
    return model_inputs

In [8]:
def train_qa_s2s_epoch(model, dataset, tokenizer, optimizer, scheduler, args, e=0, curriculum=False):
    model.train()
    # make iterator
    if curriculum:
        train_sampler = SequentialSampler(dataset)
    else:
        train_sampler = RandomSampler(dataset)
    # create more args (extra args)
    model_collate_fn = functools.partial(
        make_qa_s2s_batch, tokenizer=tokenizer, max_len=args.max_length, device="cuda:0"
        ####
    )
    data_loader = DataLoader(dataset, batch_size=args.batch_size, sampler=train_sampler, collate_fn=model_collate_fn)
    epoch_iterator = tqdm(data_loader, desc="Iteration", disable=True)
    # accumulate loss since last print
    loc_steps = 0
    loc_loss = 0.0
    st_time = time()
    for step, batch_inputs in enumerate(epoch_iterator):
        ############ NOTE
        pre_loss = model(**batch_inputs)[0]
#         print(pre_loss)
        loss = pre_loss.sum() / pre_loss.shape[0]
        loss.backward()
        # optimizer
        if step % args.backward_freq == 0:
            optimizer.step()
            scheduler.step()
            model.zero_grad()
        # some printing within the epoch
        loc_loss += loss.item()
        loc_steps += 1
        if step % args.print_freq == 0 or step == 1:
            print(
                "{:2d} {:5d} of {:5d} \t L: {:.3f} \t -- {:.3f}".format(
                    e, step, len(dataset) // args.batch_size, loc_loss / loc_steps, time() - st_time,
                )
            )
            loc_loss = 0
            loc_steps = 0


In [9]:
def train_qa_s2s(qa_s2s_model, qa_s2s_tokenizer, s2s_train_dset, s2s_valid_dset, s2s_args):
    s2s_optimizer = AdamW(qa_s2s_model.parameters(), lr=s2s_args.learning_rate, eps=1e-8)
    s2s_scheduler = get_linear_schedule_with_warmup(
        s2s_optimizer,
        num_warmup_steps=400,
        num_training_steps=(s2s_args.num_epochs + 1) * math.ceil(len(s2s_train_dset) / s2s_args.batch_size),
    )
    for e in range(s2s_args.num_epochs):
        train_qa_s2s_epoch(
            qa_s2s_model,
            s2s_train_dset,
            qa_s2s_tokenizer,
            s2s_optimizer,
            s2s_scheduler,
            s2s_args,
            e,
            curriculum=(e == 0),
        )
        m_save_dict = {
            "model": qa_s2s_model.module.state_dict()
                    if hasattr(qa_s2s_model, "module") else qa_s2s_model.state_dict(),
            "optimizer": s2s_optimizer.state_dict(),
            "scheduler": s2s_scheduler.state_dict(),
        }
        print("Saving model {}".format(s2s_args.model_save_name))
        eval_qa_s2s_epoch(qa_s2s_model, s2s_valid_dset, qa_s2s_tokenizer, s2s_args)
        torch.save(m_save_dict, "bart-base_model.pth")
    writer.flush()
    print('Done!')

In [13]:
class ELI5DatasetS2S(Dataset):
    def __init__(
        self, examples_array, make_doc_fun=None, extra_answer_threshold=3, document_cache=None, training=True
    ):
        self.training = training
        self.data = examples_array
        self.make_doc_function = make_doc_fun
            
        self.document_cache = {} if document_cache is None else document_cache
        
        assert not (make_doc_fun is None and document_cache is None)
        # make index of specific question-answer pairs from multi-answers
        # Fix: just using only one answer for training 
        # Not require self.training
        if self.training:
            self.qa_id_list = [
                (i, j)
                for i, qa in enumerate(self.data)
                for j, a in enumerate(qa["answers"])
#                 if j == 0 or sc >= extra_answer_threshold
            ]
        else:
            self.qa_id_list = [(i, 0) for i in range(len(self.data))]
#         self.qa_id_list = [(i, 0) for i in range(len(self.data))]

    def __len__(self):
        return len(self.qa_id_list)

    def make_example(self, idx):
        i, j = self.qa_id_list[idx]
        example = self.data[i]
        question = example["question"]
        # using only the first answer.
        answer = example["answers"][j]
        q_id = example["question_id"]
#         if self.make_doc_function is not None:
#             self.document_cache[q_id] = self.document_cache.get(q_id, self.make_doc_function(example["title"]))
        document = self.document_cache[q_id] 
        # return from dict document_cache { question_id : ctxs_dense}
        # Done
        in_st = "question: {} context: {}".format(
            question.lower().strip(), document.lower().strip(),
        )
        out_st = answer
        return (in_st, out_st)

    def __getitem__(self, idx):
        return self.make_example(idx)


In [11]:
def eval_qa_s2s_epoch(model, dataset, tokenizer, args):
    model.eval()
    # make iterator
    train_sampler = SequentialSampler(dataset)
    model_collate_fn = functools.partial(
        make_qa_s2s_batch, tokenizer=tokenizer, max_len=args.max_length, device="cuda:0"
    )
    data_loader = DataLoader(dataset, batch_size=args.batch_size, sampler=train_sampler, collate_fn=model_collate_fn)
    epoch_iterator = tqdm(data_loader, desc="Iteration", disable=True)
    # accumulate loss since last print
    loc_steps = 0
    loc_loss = 0.0
    st_time = time()
    with torch.no_grad():
        for step, batch_inputs in enumerate(epoch_iterator):
            pre_loss = model(**batch_inputs)[0]
            loss = pre_loss.sum() / pre_loss.shape[0]
            loc_loss += loss.item()
            loc_steps += 1
            if step % args.print_freq == 0:
                print(
                    "{:5d} of {:5d} \t L: {:.3f} \t -- {:.3f}".format(
                        step, len(dataset) // args.batch_size, loc_loss / loc_steps, time() - st_time,
                    )
                )
    print("Total \t L: {:.3f} \t -- {:.3f}".format(loc_loss / loc_steps, time() - st_time,))

In [ ]:
# def query_qa_dense_index(
#     question, qa_embedder, tokenizer, wiki_passages, wiki_index, n_results=10, min_length=20, device="cuda:0"
# ):
# #     q_rep = embed_questions_for_retrieval([question], tokenizer, qa_embedder, device=device)
# #     D, I = wiki_index.search(q_rep, 2 * n_results)
# #     res_passages = [wiki_passages[int(i)] for i in I[0]]
#     support_doc = "<P> " + " <P> ".join([p["passage_text"] for p in res_passages])
    
#     return support_doc

In [ ]:
# def format_doc(dataset):
#     eli5_valid_docs = []
#     for example in val:
#         support_doc = "<P> " + " <P> ".join([str(p) for p in example['ctxs']])
#         eli5_valid_docs += [(example['question_id'], support_doc)]
#     return eli5_valid_docs

In [ ]:
# eli5_valid_docs[0]

### Save for later load

In [ ]:
# with open(r"/kaggle/working/eli5_valid_docs.json", 'w') as f:
#     f.write(json.dumps(eli5_valid_docs))

In [ ]:
# # pre-computing support documents
# eli5_train_docs = []
# for example in train:
#     support_doc = "<P> " + " <P> ".join([str(p) for p in example['ctxs']])
#     eli5_train_docs += [(example['question_id'], support_doc)]

In [ ]:
# with open(r"/kaggle/working/eli5_train_docs.json", 'w') as f:
#     f.write(json.dumps(eli5_train_docs))

### Train

In [14]:
s2s_args = ArgumentsS2S()

eli5_train_docs = json.load(open('/kaggle/input/dataprepare/eli5_train_docs.json'))
eli5_valid_docs = json.load(open('/kaggle/input/dataprepare/eli5_valid_docs.json'))

                                ## my train and val          # question_id, doc_compute_dense
s2s_train_dset = ELI5DatasetS2S(train, document_cache=dict([(k, d) for k, d in eli5_train_docs]))
s2s_valid_dset = ELI5DatasetS2S(val, document_cache=dict([(k, d) for k, d in eli5_valid_docs]), training=False)

In [ ]:
s2s_valid_dset[0]

In [15]:
qa_s2s_tokenizer, pre_model = make_qa_s2s_model(
    model_name="facebook/bart-base",
    from_file=None,
    device="cuda:0"
)

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [ ]:
# import os
# s2s_optimizer = AdamW(qa_s2s_model.parameters(), lr=s2s_args.learning_rate, eps=1e-8)
# s2s_scheduler = get_linear_schedule_with_warmup(
# s2s_optimizer,
# num_warmup_steps=400,
# num_training_steps=(s2s_args.num_epochs + 1) * math.ceil(len(s2s_train_dset) / s2s_args.batch_size),
# )
# m_save_dict = {
#             "model": qa_s2s_model.module.state_dict()
#                     if hasattr(qa_s2s_model, "module") else qa_s2s_model.state_dict(),
#             "optimizer": s2s_optimizer.state_dict(),
#             "scheduler": s2s_scheduler.state_dict(),
#         }
# print("Saving model {}".format(s2s_args.model_save_name))
# eval_qa_s2s_epoch(qa_s2s_model, s2s_valid_dset, qa_s2s_tokenizer, s2s_args)
# torch.save(m_save_dict, "eli5_bart_model_0.pth")

In [ ]:
qa_s2s_model = torch.nn.DataParallel(pre_model)
train_qa_s2s(qa_s2s_model, qa_s2s_tokenizer, s2s_train_dset, s2s_valid_dset, s2s_args)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a ma

 0     0 of 100549 	 L: 6.134 	 -- 6.639
 0     1 of 100549 	 L: 6.339 	 -- 7.516
 0   100 of 100549 	 L: 6.096 	 -- 99.965
 0   200 of 100549 	 L: 5.534 	 -- 202.945
 0   300 of 100549 	 L: 5.074 	 -- 305.355
 0   400 of 100549 	 L: 4.827 	 -- 408.529
 0   500 of 100549 	 L: 4.632 	 -- 511.759


# Test

In [ ]:
# !pip install nlp

In [ ]:
# import nlp
# eli5 = nlp.load_dataset('eli5')

In [5]:
# generate answer from input "question: ... context: <p> ..."
def qa_s2s_generate(
    question_doc,
    qa_s2s_model,
    qa_s2s_tokenizer,
    num_answers=1,
    num_beams=None,
    min_len=64,
    max_len=256,
    do_sample=False,
    temp=1.0,
    top_p=None,
    top_k=None,
    max_input_length=512,
    device="cuda:0",
):
    model_inputs = make_qa_s2s_batch([(question_doc, "A")], qa_s2s_tokenizer, max_input_length, device=device,)
    n_beams = num_answers if num_beams is None else max(num_beams, num_answers)
    
    generated_ids = qa_s2s_model.generate(
        input_ids=model_inputs["input_ids"],
        attention_mask=model_inputs["attention_mask"],
        min_length=min_len,
        max_length=max_len,
        ###
        do_sample=do_sample,
        ###
        early_stopping=True,
        num_beams=1 if do_sample else n_beams,
        temperature=temp,
        ###
        top_k=top_k,
        top_p=top_p,
        ###
        eos_token_id=qa_s2s_tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        num_return_sequences=num_answers,
        decoder_start_token_id=qa_s2s_tokenizer.bos_token_id,
    )
    return [qa_s2s_tokenizer.decode(ans_ids, skip_special_tokens=True).strip() for ans_ids in generated_ids]

# beam 6, top p 50

## Prediction

In [ ]:
predicted = []
reference = []


# Generate answers for the full test set
for example in val:
    # create support document with the dense index
    question = example['question']
    doc = "<P> " + " <P> ".join([str(p) for p in example['ctxs']])
    # concatenate question and support document into BART input
    question_doc = "question: {} context: {}".format(question, doc)
    
    # generate an answer with beam search
    answer = qa_s2s_generate(
                            ####
            question_doc, pre_model, qa_s2s_tokenizer,
            num_answers=1,
            num_beams=8,
            min_len=96,
            max_len=256,
            max_input_length=1024,
            device="cuda:0"
    )[0]
    
    predicted += [answer]

    reference += [example['answers']]

In [ ]:
# abc = []
# abc += [val[0]['answers']]
# abc+= [val[1]['answers']]
# abc

In [3]:
with open(r"/kaggle/working/pred_ref_t5.json", 'w') as f:
    f.write(json.dumps([predicted, reference]))

NameError: name 'predicted' is not defined

## Evaluation

In [1]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=12f6d46b18cdcbeb308f4e2a1d93cbe998f38b23835145bbb00258f350edf6c4
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install nlp

In [9]:
import json
import nlp

In [35]:
with open('/kaggle/input/eval-t5-2/pred_ref_t5.json', 'r')  as f:
    pred_ref_t5 = json.load(f)

In [36]:
predicted_load = pred_ref_t5[0]
reference_load = pred_ref_t5[1]
print(len(predicted_load), len(reference_load))

1507 1507


In [37]:
reference_load[4][0]

'Cannabis is a depressant drug, which means it slows down messages travelling between your brain and body. When large doses of cannabis are taken, it may also produce hallucinogenic effects. Cannabis can cause: reduced coordination, slower reaction times, slower information processing, confusion, changes in vision, hearing, and time and space perception. A person who has been using cannabis may think that they will be able to drive safely. However, the cannabis may have affected their view and experience of reality, and their judgement. Their actions and responses may be quite different to what is actually needed, but they may not be aware of how much their driving skills have been affected. Even after a small amount of cannabis you should not drive for at least 5 hours. (From:  URL_0 )'

In [38]:
reference_load_ans = [reference_load[i][0] for i in range(len(reference_load))]

In [39]:
reference_load_ans[1]

'It is water vapor and ice. They are produced from the hot engine exhaust in the cold atmosphere. Water vapor from the engine exhaust mixed with unburnt particulate in the jet fuel gives the surrounding moist air something to latch onto and ice crystals form. Depending on the hight of the aircraft, they can last seconds to hours. If you have seen a running car on a brisk morning, that is a similar effect. The car is too close to the relatively warmer ground that trails do not last for more than a second.'

In [14]:
import nlp
nlp_rouge = nlp.load_metric('rouge')

Downloading:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

In [40]:
scores = nlp_rouge.compute(
    predicted_load, reference_load_ans,
    rouge_types=['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
    use_agregator=False, use_stemmer=False
)
# df = pd.DataFrame({
#     'rouge1': [scores['rouge1'].mid.precision, scores['rouge1'].mid.recall, scores['rouge1'].mid.fmeasure],
#     'rouge2': [scores['rouge2'].mid.precision, scores['rouge2'].mid.recall, scores['rouge2'].mid.fmeasure],
#     'rougeL': [scores['rougeL'].mid.precision, scores['rougeL'].mid.recall, scores['rougeL'].mid.fmeasure],
# }, index=[ 'P', 'R', 'F'])
# df.style.format({'rouge1': "{:.4f}", 'rouge2': "{:.4f}", 'rougeL': "{:.4f}"})
scores['rougeL'][0:5]

[Score(precision=0.27631578947368424, recall=0.0846774193548387, fmeasure=0.12962962962962965),
 Score(precision=0.14864864864864866, recall=0.11702127659574468, fmeasure=0.13095238095238096),
 Score(precision=0.12307692307692308, recall=0.08695652173913043, fmeasure=0.1019108280254777),
 Score(precision=0.13846153846153847, recall=0.11842105263157894, fmeasure=0.12765957446808512),
 Score(precision=0.18840579710144928, recall=0.1, fmeasure=0.13065326633165827)]

In [47]:
import pandas as pd
rougeL = pd.DataFrame(scores['rougeL'])
top10 = rougeL.sort_values(by=["fmeasure"],ascending=True)[0:10]

In [48]:
idx_top_best_10 = list(top10.index)
idx_top_best_10

[1232, 223, 802, 912, 679, 991, 330, 421, 1479, 1082]

In [46]:
for i in  idx_top_best_10:
    print((predicted_load[i], reference_load_ans[i]))

("_URL_0_ The rovers don't have to be directed to the suspected source, but they can't confirm or deny any speculations. They aren't going to be able to confirm that there is water on Mars, but it's not likely that there's a source of water on the surface of Mars. It's also possible that a rover may have carried the bacteria Bacillus safensis to Mars.", " >  why can’t one of the rovers be directed to the suspected source and simply confirm or deny speculations? Rovers are extremely slow, you can't just drive them to wherever you want. [Opportunity travelled 45km in all its 12 years on Mars]( URL_0 ) We know (for a long time now) that there is indeed [water (ice) on Mars]( URL_1 ), no need to send a probe there just to confirm it.")
('If you were on the moon during a total lunar eclipse, you would see the Moon from an angle that is about 2 degrees different from the angle of sight of an observer who sees the Moon on the western horizon. You would see a lunar eclipse when you look up at 

In [18]:
!pip install rouge

In [25]:
from nltk import PorterStemmer
from rouge import Rouge
from spacy.lang.en import English
from time import time
from spacy.tokenizer import Tokenizer
import pandas as pd

stemmer = PorterStemmer()
rouge = Rouge()
nlp = English()
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
# tokenizer = nlp.Defaults.create_tokenizer(nlp)

def compute_rouge_eli5(compare_list):
    preds = [" ".join([stemmer.stem(str(w))
                       for w in tokenizer(pred)])
             for gold, pred in compare_list]
    golds = [" ".join([stemmer.stem(str(w))
                       for w in tokenizer(gold)])
             for gold, pred in compare_list]
    scores = rouge.get_scores(preds, golds, avg=True)
    return scores
    

compare_list = [(g, p) for p, g in zip(predicted_load, reference_load_ans)]
scores = compute_rouge_eli5(compare_list)
df = pd.DataFrame({
    'rouge1': [scores['rouge-1']['p'], scores['rouge-1']['r'], scores['rouge-1']['f']],
    'rouge2': [scores['rouge-2']['p'], scores['rouge-2']['r'], scores['rouge-2']['f']],
    'rougeL': [scores['rouge-l']['p'], scores['rouge-l']['r'], scores['rouge-l']['f']],
}, index=[ 'P', 'R', 'F'])
df.style.format({'rouge1': "{:.4f}", 'rouge2': "{:.4f}", 'rougeL': "{:.4f}"})

,rouge1,rouge2,rougeL
P,0.3150,0.0535,0.2831
R,0.2172,0.0380,0.1935
F,0.2288,0.0376,0.2043


In [ ]:
# idx = 100
# print("Question:   {}".format(val[idx]['question']))
# print("Prediction: {}".format(predicted_load[idx]))
# print("Reference:  {}".format(reference_load[idx]))

In [ ]:
# import random

In [2]:
!pip -q install psycopg2-binary 
!pip -q install transformers
!pip -q install python-dotenv

In [ ]:
import os
import dotenv
import psycopg2
import time
from transformers import  AutoModel, AutoTokenizer

# from src.utils import *
# from src.train_utils import *

dotenv.load_dotenv()

DBNAME=os.getenv("DBNAME", "wiki_db")
HOST=os.getenv("HOST", "124.158.12.207")
PORT=os.getenv("PORT", "15433")
USER=os.getenv("USER", "gradlab")
PWD=os.getenv("PASSWORD", "baldarg")
# TB_CLIENT=os.getenv("TB_CLIENT","client_tb")
TB_WIKI=os.getenv("TB_WIKI", "wiki_tb")
# MSD_WIKI = bool(os.getenv("MSD_WIKI", False))

#TODO: Use this function

def get_embds_qs(question, tokenizer, model, device):
    # Tokenize sentences
    q_toks = tokenizer.batch_encode_plus(question, max_length=128, pad_to_max_length=True)
    q_ids, q_mask = (
        torch.LongTensor(q_toks["input_ids"]).to(device),
        torch.LongTensor(q_toks["attention_mask"]).to(device),
    )
    with torch.no_grad():
        q_reps = model.embed_questions(q_ids, q_mask).cpu().type(torch.float)

    return q_reps

def query_embd(embd, limit_doc=3, ):
    embd = str(list(embd.cpu().detach().numpy().reshape(-1)))
    try:
        connection = psycopg2.connect(dbname=DBNAME,host=HOST,port=PORT,user=USER,password=PWD)
        cursor = connection.cursor()
        aemb_sql = f"""
                        SET LOCAL ivfflat.probes = 3;
                        SELECT content 
                        FROM {TB_WIKI}
                        ORDER BY embedd <#> %s LIMIT %s;
                    """
        cursor.execute(aemb_sql,(embd, limit_doc))
        connection.commit()
        rows = cursor.fetchall()

        if connection: 
            cursor.close()
            connection.close()
        
        return rows
        
    except (Exception, psycopg2.Error) as error: 
        print("Failed query record from database {}".format(error))

# def load_model_qs(
#     pretrain_name= r"vblagoje/dpr-question_encoder-single-lfqa-wiki", 
#     device = torch.device("cuda:0")
#     ):
#     qs_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(pretrain_name)
#     qs_model = DPRQuestionEncoder.from_pretrained(pretrain_name)
#     qs_model.to(device)
    
#     return qs_model, qs_tokenizer


def retrieve(question, qs_embedder, qs_tokenizer, device, limit_doc=10):
    # question_embd = get_ embds_qs(qs_embedder, qs_tokenizer, question, device=device)
    question_embd = get_embds_qs([question], qs_tokenizer, qs_embedder, device=device)
    documents_wiki = query_embd(question_embd, limit_doc=limit_doc)
    return [doc[-1] for doc in documents_wiki]

if __name__ == "__main__":

    qar_tokenizer = AutoTokenizer.from_pretrained('yjernite/retribert-base-uncased')
    qar_model = AutoModel.from_pretrained('yjernite/retribert-base-uncased').to('cuda:0')
    _ = qar_model.eval()

    qa_s2s_tokenizer, pre_model = make_qa_s2s_model(model_name="t5-small",
                                                from_file= r"/kaggle/input/model-t5/t5-small_model.pth",
                                                device="cuda:0")

    while(True):
        question = input("\nUSER:")
        if question == "[EXIT]":
            break
        else:   
            doc_10 = retrieve(question, qar_model, qar_tokenizer, "cuda:0", limit_doc=10)
            doc = "<P> " + " <P> ".join([p for p in doc_10])
            question_doc = "question: {} context: {}".format(question, doc)

            # generate an answer with beam search
            # start = time.time()
            answer = qa_s2s_generate(
                    question_doc, pre_model, qa_s2s_tokenizer,
                    num_answers=1,
                    num_beams=6,
                    min_len=3,
                    max_len=100,
                    max_input_length=1024,
                    device="cuda:0")[0]       
            # end = time.time()

            print("\nBOT:", answer.replace("\n", ""))
            # print("time to generate answer: {}".format(end-start))


Some weights of RetriBertModel were not initialized from the model checkpoint at yjernite/retribert-base-uncased and are newly initialized: ['bert_query.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


...


BOT: China is the largest cheese producer in the world. It's the largest producer of cheese, and it's not a big deal.



USER: What is the largest country in Africa?



BOT: Britain is the largest country in Africa. It's a big country in the world, and it's the largest nation in Africa, and the largest in Africa is the world's largest country.



USER: What is the currency in South Africa?



BOT: China is the currency in South Africa. It's not a currency, but it's the currency of South Africa, which is a foreign currency.
